# Simulazione modello AIM

In [1]:
import numpy as np
import numba
from numba import njit, prange
from scipy.io import savemat
import matplotlib.pyplot as plt
from simula import *

Ho un po' di funzioni che fanno evoluzione temporale, scrivo qua sotto per non dimenticare:
- evoluzione_profili = profili e basta solo alla fine (non ha senso usarla)
- evoluzione_filmino = salva densità e magnetizzazione che sono effettivamente da controllare, secondo quello che dice array
- evoluzione_track = tiene traccia casualmente di alcune particelle per osservare meglio la dinamica
- evoluzione_scorporata = scorpora il meccanismo di flip da quello della diffusione

Compilazioni preliminari per velocizzare

In [2]:
Lx = 5
Ly = 5
n = 200
particles = np.empty((n,3), dtype=np.int64)
particles[:,0] = np.random.randint(0, Lx, size=n)
particles[:,1] = np.random.randint(0, Ly, size=n)
particles[:,2] = 2*np.random.randint(0,2,size=n) - 1

density = np.zeros((Ly, Lx), np.int32)
magnet  = np.zeros((Ly, Lx), np.int32)

# popola matrici iniziali
for k in range(n):
    x, y, s = particles[k]
    density[y, x] += 1
    magnet[y, x]  += s

array = np.arange(0,0)
rateflip_numba(Lx,Ly,1,1,1,1,1,1,magnet,density)
evoluzione_profili(Lx,Ly,1,1,1,particles,1,1,1)
evoluzione_filmino(Lx,Ly,1,1,1,particles,10,1,1,array) #compila

(array([], shape=(0, 5, 5), dtype=float64),
 array([], shape=(0, 5, 5), dtype=float64),
 array([-0.08, -0.06, -0.06, -0.11, -0.09, -0.11, -0.09, -0.14, -0.09,
        -0.16]))

In [3]:
# Parametri delle simulazioni
Lx, Ly      = [50, 70], [50, 70]
D, epsilon  = 1.0, 0.9
t_max       = 6_000_000
freq_frames = 5_000
num = 11 #numero di simulazioni da lanciare per ogni taglia

array = build_log_indices(t_max, freq_frames) 
array = np.array(array, dtype=np.int64) 

#array = np.arange(0, t_max, freq_frames, dtype=np.int64) # per test

dens_init = np.array([5, 5]) # ci sono cluster a basse T?
T         = np.array([0.2, 0.2]) #temperature
betas     = 1/T
gammas    = np.array([0, 0]) # grado di località
metodi    = np.array([1, 1]) # metodo di "località"


for j in range(1,num):
    for i, beta in enumerate(betas):
        n = int(Lx[i] * Ly[i] * dens_init[i]) #n_particelle data densità
    
        # genera particelle dato n
        particles = np.empty((n,3), dtype=np.int64)
        particles[:,0] = np.random.randint(0, Lx[i], size=n)
        particles[:,1] = np.random.randint(0, Ly[i], size=n)
        particles[:,2] = 2*np.random.randint(0,2,size=n) - 1

        # chiama la funzione numba (già compilata)
        frames_d, frames_m, order_param = evoluzione_filmino(
            Lx[i], Ly[i], D, epsilon,
            beta, particles, t_max,
            gammas[i], metodi[i], array
        )

        # salvataggio per successiva visualizzazione (anche in Matlab)

        savemat(f'{j}stabL{Lx[i]:.0f}' + 'frames_d'+'.mat', {'frames_d': frames_d})
        savemat(f'{j}stabL{Lx[i]:.0f}' + 'frames_m'+'.mat', {'frames_m': frames_m})
        savemat(f'{j}stabL{Lx[i]:.0f}' + 'order_param'+'.mat', {'order_param': order_param})